#BRAND LOGO DETECTION

##IMPORTING LIBRARIES

In [ ]:
%%capture
!pip install tensorflow-addons

In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import PIL

import tensorflow as tf

from tensorflow.keras.layers import *
import tensorflow_addons as tfa

from sklearn.metrics import accuracy_score

In [ ]:
BATCH_SIZE = 8
HEIGHT,WIDTH = 224,224
CHANNELS = 3
NUM_CLASSES =6 
SEED = 143

##LOADING DATASET

In [ ]:

from google_drive_downloader import GoogleDriveDownloader as gdd

def download(flid,path,unzp=False):
  return gdd.download_file_from_google_drive(file_id=flid,
                                    dest_path=path,
                                    unzip=unzp)




##Download the video file

In [ ]:
# https://drive.google.com/file/d/1z1_Bgthqy2RaU8Yvdcscqul4DbQdoAFD/view?usp=sharing
# https://drive.google.com/file/d/1KwK5YdfaYJQT9CINhLa8g9oQtZawUCOP/view?usp=sharing
# https://drive.google.com/file/d/13gX_hg3903OKBviXVur5i7VycOlW-0nW/view?usp=sharing

download('13gX_hg3903OKBviXVur5i7VycOlW-0nW','./content/test_video/test_vid.mp4',False)

##VIDEO TO FRAMES

In [ ]:
import cv2


def vid_to_frames(video_path):
    
  # Read the video from specified path
  cam = cv2.VideoCapture(video_path)
    #../content/content/test_vid.mp4 
  try:
        
      # creating a folder named data
      if not os.path.exists('test/test_data'):
          os.makedirs('test/test_data')

    
  # if not created then raise error
  except OSError:
      print ('Error: Creating directory of data')
    
  
  currentframe = 0
  os.chdir('/content/test/test_data')  
  while(True):
        
      
      ret,frame = cam.read()
    
      if ret:
          
          name = '../test_data/frame' + str(currentframe) + '.jpg'
          cv2.imwrite(name, frame)
          currentframe += 1
      else:
          break
  cam.release()
  cv2.destroyAllWindows()
  %cd /content/


In [ ]:
vid_to_frames('../content/content/test_video/test_vid.mp4')

/content


### local test set

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 0.2, 
    zoom_range = 0.2,
   )
my_test = test_datagen.flow_from_directory(
                             '/content/test',
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "categorical",
                             shuffle = False,
                             seed = SEED,
                            #  classes=CLASSES,
                             color_mode='rgb'
                            )

Found 1500 images belonging to 1 classes.


##TRAIN DATASET PREPROCESSING

In [ ]:
gdd.download_file_from_google_drive(file_id='1f0UrrqcAMyZk4WjQFt7l1W0z_O0-LTqt',
                                    dest_path='./content/brands.zip',
                                    unzip=True)

Unzipping...Done.


In [ ]:
TRAIN_PATH = "../content/content/logos3/train"
TEST_PATH = "../content/content/logos3/test"


CLASSES=['Burger King','KFC','McDonalds','Other','Starbucks','Subway']

In [ ]:
from tensorflow.keras.applications.densenet import preprocess_input 

In [ ]:
#Data generated
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    horizontal_flip=True,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    shear_range = 0.2, 
    zoom_range = 0.21,
    validation_split=0.12,  
    preprocessing_function= tf.keras.applications.nasnet.preprocess_input 
   )

#Data stored in train & test 
train_ds = train_datagen.flow_from_directory(
                             TRAIN_PATH,
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "categorical",
                             shuffle = True,
                             seed = SEED,
                             classes=CLASSES,
                             subset = "training",
                             color_mode='rgb'
                            )
val_ds = train_datagen.flow_from_directory(
                             TRAIN_PATH,
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "categorical",
                             shuffle = False,
                             seed = SEED,
                             classes=CLASSES,
                             subset = "validation",
                             color_mode='rgb'
                            )


classes_dict = train_ds.class_indices
# classes_dict

Found 1533 images belonging to 6 classes.
Found 205 images belonging to 6 classes.


##TEST DATASET PREPROCESSING

In [ ]:

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    horizontal_flip=True,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    shear_range = 0.2, 
    zoom_range = 0.2,
    preprocessing_function=tf.keras.applications.nasnet.preprocess_input 
   )
test_ds = test_datagen.flow_from_directory(
                             TEST_PATH,
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "categorical",
                             shuffle = False,
                             seed = SEED,
                             classes=CLASSES,
                             color_mode='rgb'
                            )

Found 560 images belonging to 6 classes.


## VGG16

In [ ]:
def create_model7():
    model = tf.keras.applications.VGG16(weights= "imagenet",
                                    include_top=False,
                                    input_shape=(HEIGHT,WIDTH,CHANNELS), pooling="avg")
    predictions = tf.keras.layers.Dense(32, activation='relu', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    
    
    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(512, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    # x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(6, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    for layer in model.layers[:-10]:
        layer.trainble = False
    return model


model=create_model7()  

58892288/58889256 [==============================] - 0s 0us/step


## InceptionResNetV2

In [ ]:

def create_model8():
    model = tf.keras.applications.InceptionResNetV2(weights= "imagenet",
                                    include_top=False,
                                    input_shape=(HEIGHT,WIDTH,CHANNELS), pooling="avg")
    predictions = tf.keras.layers.Dense(32, activation='relu', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    
    
    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(512, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    # x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    # x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(6, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    for layer in model.layers[:-2]:
        layer.trainble = False
    return model


model=create_model8()  

219062272/219055592 [==============================] - 7s 0us/step


## BASE MODEL

In [ ]:

IMAGE_SIZE=(224,224)
def conv_block(filters, inputs):
    x = tf.keras.layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(inputs)
    x = tf.keras.layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = tf.keras.layers.MaxPool2D()(x)
    return outputs


def dense_block(units, dropout_rate, inputs):
    x = tf.keras.layers.Dense(units, activation="relu")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = tf.keras.layers.Dropout(dropout_rate)(x)

    return outputs
def build_model():
    inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
    x = tf.keras.layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.MaxPool2D()(x)

    x = conv_block(32, x)
    x = conv_block(64, x)

    x = conv_block(128, x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = conv_block(256, x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Flatten()(x)
    x = dense_block(512, 0.7, x)
    x = dense_block(128, 0.5, x)
    x = dense_block(64, 0.3, x)

    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model
   
def create_model():
    model=build_model()
    model = tf.keras.Model(model.input, model.layers[-2].output)
    outputs = tf.keras.layers.Dense(NUM_CLASSES,activation = "softmax", name = "output")(model.output)
    
    model = tf.keras.Model(model.input, outputs)
    return model

model=create_model()


##MOBILENET

In [ ]:

def create_model3():
    model = tf.keras.applications.MobileNetV2(weights= "imagenet",
                                                     include_top=False,
                                                     input_shape=(HEIGHT,WIDTH,CHANNELS))
    predictions = tf.keras.layers.Dense(32, activation='sigmoid', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)

    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(512, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    # x = tf.keras.layers.Dense(128, activation = "relu")(x)
    # x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(6, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    # for layer in model.layers[:-14]:
    #     layer.trainble = False
    return model



# model2 = create_model3()


## CUSTOM KERAS MODEL 

In [ ]:

IMAGE_SIZE=(224,224)
def conv_block(filters, inputs):
    x = tf.keras.layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(inputs)
    x = tf.keras.layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = tf.keras.layers.MaxPool2D()(x)

    return outputs


def dense_block(units, dropout_rate, inputs):
    x = tf.keras.layers.Dense(units, activation="relu")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = tf.keras.layers.Dropout(dropout_rate)(x)

    return outputs
def build_model():
    inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
    x = tf.keras.layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.MaxPool2D()(x)

    x = conv_block(32, x)
    x = conv_block(64, x)

    x = conv_block(128, x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = conv_block(256, x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Flatten()(x)
    x = dense_block(512, 0.7, x)
    x = dense_block(128, 0.5, x)
    x = dense_block(64, 0.3, x)

    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model
   
def create_model2():
    model=build_model()
    # model = tf.keras.models.load_model("../input/pneumonia-classification-challenge/xray_model.h5")
    model = tf.keras.Model(model.input, model.layers[-2].output)
    outputs = tf.keras.layers.Dense(6,activation = "softmax", name = "output")(model.output)
    
    model = tf.keras.Model(model.input, outputs)
    return model
tf.keras.applications.DenseNet121
print("Keras Model")


Keras Model


## DENSENET121

In [ ]:
#best_score
def create_model():
    model = tf.keras.applications.DenseNet121(weights= "imagenet",
                                    include_top=False,
                                    input_shape=(HEIGHT,WIDTH,CHANNELS), pooling="avg")
    predictions = tf.keras.layers.Dense(32, activation='sigmoid', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    
    # model.load_weights("../content/best_model.h5")
    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(512, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    # x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(6, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    for layer in model.layers[:-14]:
        layer.trainble = False
    return model

model = create_model()


29097984/29084464 [==============================] - 0s 0us/step


##LOADING SAVED MODEL

In [ ]:
# https://drive.google.com/file/d/15rxX48a0qJaEwiFbGQ6PLMX3pr5hdEak/view?usp=sharing
# https://drive.google.com/file/d/1FItPjTndNgunasm0FWCFQDr3qJ1WfHf6/view?usp=sharing
# https://drive.google.com/file/d/10vxQ7afAJresvSZh5EPCGQ6-4JWK3CZm/view?usp=sharing
# 1aVuN4xVwiPazCxH3KK4sW14y6HS6lAK5

download(flid='10vxQ7afAJresvSZh5EPCGQ6-4JWK3CZm',
         path='/content/saved_model/best_model_74.h5'
         )

In [ ]:
from tensorflow.keras.models import load_model
model=load_model('/content/saved_model/best_model_74.h5')

##COMPILING MODEL

In [ ]:
def compile_model(model, lr):
    
    optimizer =tf.keras.optimizers.Adam(learning_rate=lr)
        
    loss = "categorical_crossentropy"
    
    metrics = [
       tfa.metrics.F1Score(num_classes = NUM_CLASSES,average = "macro", name = "f1_score"),
       tf.keras.metrics.CategoricalAccuracy(name='acc')
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [ ]:
import datetime, os
%reload_ext tensorboard

METRIC = "val_f1_score"

def create_callbacks(metric = METRIC): 
    
    cpk_path = './best_model.h5'
    
    #Saving model checkpoints
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor= metric,
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    # Reducing learning rate
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= metric,
        mode='max',
        factor=0.1,
        patience=1,
        verbose=1
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= metric,
        mode='max',
        patience=10, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop,tensorboard_callback]         
    
    return callbacks

## MODEL EVALUATION

In [ ]:
EPOCHS= 10
VERBOSE =1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
  
  
  model = compile_model(model, lr=0.0001)

  callbacks = create_callbacks()

  history = model.fit(train_ds, 
                      epochs=EPOCHS,
                      callbacks=callbacks,
                      validation_data = val_ds,
                        verbose=VERBOSE)

Epoch 1/10
192/192 [==============================] - 74s 304ms/step - loss: 0.8948 - f1_score: 0.4443 - acc: 0.6895 - val_loss: 2.0432 - val_f1_score: 0.1086 - val_acc: 0.4829

Epoch 00001: val_f1_score improved from -inf to 0.10855, saving model to ./best_model.h5
Epoch 2/10
192/192 [==============================] - 52s 271ms/step - loss: 0.2399 - f1_score: 0.8635 - acc: 0.9302 - val_loss: 3.7153 - val_f1_score: 0.0985 - val_acc: 0.2293

Epoch 00002: val_f1_score did not improve from 0.10855

Epoch 00002: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 3/10
192/192 [==============================] - 52s 272ms/step - loss: 0.1383 - f1_score: 0.9541 - acc: 0.9687 - val_loss: 1.8661 - val_f1_score: 0.2604 - val_acc: 0.5854

Epoch 00003: val_f1_score improved from 0.10855 to 0.26041, saving model to ./best_model.h5
Epoch 4/10
192/192 [==============================] - 52s 271ms/step - loss: 0.0876 - f1_score: 0.9704 - acc: 0.9804 - val_loss: 0.6633 - val_f1_scor

## Test data from Video

In [ ]:

pred = model.predict(my_test)
labels = (train_ds.class_indices)
predicted_class_indices=np.argmax(pred,axis=1)
labels = dict((v,k) for k,v in labels.items())

predictedLables= [labels[k] for k in predicted_class_indices]

In [ ]:

results=pd.DataFrame(predictedLables)
results.to_csv("results.txt",index=False,header=False)

##PREDICTING TEST DATA

In [ ]:
pred = model.predict(test_ds)
labels = (train_ds.class_indices)
predicted_class_indices=np.argmax(pred,axis=1)
labels = dict((v,k) for k,v in labels.items())

predictedLables= [labels[k] for k in predicted_class_indices]

actualLables= [labels[k] for k in test_ds.classes]



In [ ]:
accuracy_score(actualLables,predictedLables)

0.7214285714285714

In [ ]:
filenames=test_ds.filenames

directory= test_ds.directory
results=pd.DataFrame({"Actual": actualLables,
                      "Predictions":predictedLables,
                      "Directory":directory
                      })
# results=pd.DataFrame(predictedLables)
results.to_csv("results.txt",index=False,header=False)

In [ ]:
results.head()

,Actual,Predictions,Directory
0,Burger King,Burger King,../content/content/logos3/test
1,Burger King,Burger King,../content/content/logos3/test
2,Burger King,Burger King,../content/content/logos3/test
3,Burger King,Burger King,../content/content/logos3/test
4,Burger King,Burger King,../content/content/logos3/test
